In [13]:
import tkinter as tk
from tkinter import messagebox
from decimal import Decimal, InvalidOperation
import re

class CalculatorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Калькулятор")
        self.root.geometry("500x700")
        self.root.resizable(False, False)
        
        # Минимальное и максимальное значения
        self.MIN_VALUE = Decimal('-1000000000000.000000')
        self.MAX_VALUE = Decimal('1000000000000.000000')
        
        # Переменные для хранения данных
        self.num1_var = tk.StringVar()
        self.num2_var = tk.StringVar()
        self.result_var = tk.StringVar()
        self.operation_var = tk.StringVar(value="+")
        
        self.setup_ui()
        self.setup_bindings()
        
    def setup_ui(self):
        # Заголовок
        title_label = tk.Label(
            self.root, 
            text="КАЛЬКУЛЯТОР", 
            font=("Arial", 20, "bold"),
            fg="blue"
        )
        title_label.pack(pady=10)
        
        # Информация о студенте
        info_frame = tk.Frame(self.root)
        info_frame.pack(pady=10)
        
        student_info = [
            "ФИО: Ломова Виктория Александровна",
            "4 курс",
            "4 группа", 
            "2025 год"
        ]
        
        for info in student_info:
            label = tk.Label(
                info_frame, 
                text=info,
                font=("Arial", 12),
                anchor="w"
            )
            label.pack(pady=2)
        
        # Основной фрейм для ввода
        main_frame = tk.Frame(self.root)
        main_frame.pack(pady=20, padx=20, fill="both")
        
        # Поле для первого числа
        tk.Label(main_frame, text="Первое число:", font=("Arial", 11)).grid(
            row=0, column=0, sticky="w", pady=5
        )
        self.num1_entry = tk.Entry(
            main_frame, 
            textvariable=self.num1_var,
            font=("Arial", 12),
            width=25
        )
        self.num1_entry.grid(row=0, column=1, padx=10, pady=5)
        self.num1_entry.bind('<KeyRelease>', lambda e: self.validate_input(self.num1_entry))
        
        # Поле для второго числа
        tk.Label(main_frame, text="Второе число:", font=("Arial", 11)).grid(
            row=1, column=0, sticky="w", pady=5
        )
        self.num2_entry = tk.Entry(
            main_frame, 
            textvariable=self.num2_var,
            font=("Arial", 12),
            width=25
        )
        self.num2_entry.grid(row=1, column=1, padx=10, pady=5)
        self.num2_entry.bind('<KeyRelease>', lambda e: self.validate_input(self.num2_entry))
        
        # Выбор операции
        tk.Label(main_frame, text="Операция:", font=("Arial", 11)).grid(
            row=2, column=0, sticky="w", pady=10
        )
        
        operation_frame = tk.Frame(main_frame)
        operation_frame.grid(row=2, column=1, padx=10, pady=10, sticky="w")
        
        tk.Radiobutton(
            operation_frame,
            text="Сложение (+)",
            variable=self.operation_var,
            value="+",
            font=("Arial", 11)
        ).pack(side="left", padx=10)
        
        tk.Radiobutton(
            operation_frame,
            text="Вычитание (-)",
            variable=self.operation_var,
            value="-",
            font=("Arial", 11)
        ).pack(side="left", padx=10)
        
        # Кнопки
        button_frame = tk.Frame(self.root)
        button_frame.pack(pady=20)
        
        tk.Button(
            button_frame,
            text="Вычислить",
            command=self.calculate,
            font=("Arial", 12, "bold"),
            bg="lightgreen",
            padx=20,
            pady=10
        ).pack(side="left", padx=10)
        
        tk.Button(
            button_frame,
            text="Очистить",
            command=self.clear_all,
            font=("Arial", 12),
            bg="lightcoral",
            padx=20,
            pady=10
        ).pack(side="left", padx=10)
        
        # Результат
        result_frame = tk.Frame(self.root)
        result_frame.pack(pady=20, padx=20, fill="x")
        
        tk.Label(result_frame, text="Результат:", font=("Arial", 12, "bold")).pack(anchor="w")
        
        self.result_label = tk.Label(
            result_frame,
            textvariable=self.result_var,
            font=("Arial", 14, "bold"),
            bg="lightyellow",
            relief="solid",
            width=30,
            height=2
        )
        self.result_label.pack(pady=10, fill="x")
        
        # Инструкция
        instruction_frame = tk.Frame(self.root)
        instruction_frame.pack(pady=10, padx=20)
        
        instructions = [
            "Инструкция:",
            "• Используйте Ctrl+C и Ctrl+V для копирования/вставки",
            "• Работает как в английской, так и в русской раскладке",
            "• Поддерживаются как точка, так и запятая",
            "• Диапазон: от -1,000,000,000,000.000000 до +1,000,000,000,000.000000",
            "• Числа остаются в полях после вычисления"
        ]
        
        for instruction in instructions:
            label = tk.Label(
                instruction_frame,
                text=instruction,
                font=("Arial", 9),
                anchor="w",
                justify="left"
            )
            label.pack(anchor="w")
    
    def setup_bindings(self):
        """Настройка привязок клавиш"""
        # Биндинг на уровне виджета для Ctrl+C/Ctrl+V
        for widget in [self.num1_entry, self.num2_entry]:
            widget.bind('<Control-KeyPress>', self.handle_ctrl_key)
    
    def handle_ctrl_key(self, event):
        """Обработка Ctrl+Клавиша независимо от раскладки"""
        # Получаем код клавиши
        keycode = event.keycode
        
        # Список кодов клавиш для букв (и латинских, и русских)
        # Код 67 = C (латинская) / С (русская)
        # Код 86 = V (латинская) / М (русская)
        if event.state & 0x4:  # Проверяем, что Ctrl нажат (0x4 = Control)
            if keycode == 67:  # C/С
                self.copy_to_clipboard(event.widget)
                return "break"
            elif keycode == 86:  # V/М
                self.paste_from_clipboard(event.widget)
                return "break"
    
    def copy_to_clipboard(self, widget):
        """Копирование текста в буфер обмена"""
        if widget.selection_present():
            selected_text = widget.selection_get()
            self.root.clipboard_clear()
            self.root.clipboard_append(selected_text)
    
    def paste_from_clipboard(self, widget):
        """Вставка текста из буфера обмена с очисткой"""
        try:
            clipboard_text = self.root.clipboard_get()
        except:
            return
        
        # Определяем, какое поле редактируется
        if widget == self.num1_entry:
            var = self.num1_var
        else:
            var = self.num2_var
        
        # Очистка вставляемого текста
        cleaned = re.sub(r'[^\d\-,.]', '', clipboard_text)
        cleaned = cleaned.replace(',', '.')
        
        if cleaned.count('-') > 1:
            cleaned = cleaned[0] + cleaned[1:].replace('-', '')
        
        if cleaned.count('.') > 1:
            parts = cleaned.split('.')
            cleaned = parts[0] + '.' + ''.join(parts[1:])
        
        # Вставка текста
        current_text = var.get()
        if widget.selection_present():
            # Заменяем выделенный текст
            start = widget.index("sel.first")
            end = widget.index("sel.last")
            new_text = current_text[:start] + cleaned + current_text[end:]
        else:
            # Вставляем в позицию курсора
            cursor_pos = widget.index(tk.INSERT)
            new_text = current_text[:cursor_pos] + cleaned + current_text[cursor_pos:]
        
        var.set(new_text)
        
        # Вызываем валидацию
        if widget == self.num1_entry:
            self.validate_input(self.num1_entry)
        else:
            self.validate_input(self.num2_entry)
        
        # Ставим курсор после вставленного текста
        if widget.selection_present():
            new_cursor_pos = widget.index("sel.first") + len(cleaned)
        else:
            new_cursor_pos = cursor_pos + len(cleaned)
        widget.icursor(new_cursor_pos)
    
    def validate_input(self, entry_widget):
        """Валидация ввода в реальном времени"""
        text = entry_widget.get()
        if not text:
            return
        
        # Заменяем запятую на точку
        text = text.replace(',', '.')
        
        # Проверяем на несколько минусов
        if text.count('-') > 1:
            text = text[0] + text[1:].replace('-', '')
        
        # Проверяем на несколько точек/запятых
        if text.count('.') > 1:
            parts = text.split('.')
            text = parts[0] + '.' + ''.join(parts[1:])
        
        # Удаляем все символы, кроме цифр, минуса и точки
        cleaned_text = ''
        minus_count = 0
        dot_count = 0
        
        for i, char in enumerate(text):
            if char == '-':
                if i == 0 and minus_count == 0:
                    cleaned_text += char
                    minus_count += 1
            elif char == '.':
                if dot_count == 0:
                    cleaned_text += char
                    dot_count += 1
            elif char.isdigit():
                cleaned_text += char
        
        # Проверяем длину целой и дробной части
        if '.' in cleaned_text:
            integer_part, fractional_part = cleaned_text.split('.')
            if integer_part.lstrip('-'):
                if len(integer_part.lstrip('-')) > 13:
                    integer_part = integer_part[:1] + integer_part[1:14] if integer_part.startswith('-') else integer_part[:13]
            if len(fractional_part) > 6:
                fractional_part = fractional_part[:6]
            cleaned_text = integer_part + '.' + fractional_part
        else:
            if cleaned_text.lstrip('-'):
                if len(cleaned_text.lstrip('-')) > 13:
                    cleaned_text = cleaned_text[:1] + cleaned_text[1:14] if cleaned_text.startswith('-') else cleaned_text[:13]
        
        # Обновляем значение, если оно изменилось
        if cleaned_text != entry_widget.get():
            if entry_widget == self.num1_entry:
                self.num1_var.set(cleaned_text)
            else:
                self.num2_var.set(cleaned_text)
    
    def parse_number(self, num_str):
        """Парсинг числа из строки"""
        if not num_str:
            return None
        
        # Заменяем запятую на точку
        num_str = num_str.replace(',', '.')
        
        try:
            # Проверяем, что строка является валидным числом
            if not re.match(r'^-?\d*\.?\d*$', num_str):
                return None
            
            # Убираем лишние символы
            num_str = num_str.strip()
            
            # Проверяем случаи типа ".123" или "-.123"
            if num_str.startswith('.') or num_str.startswith('-.'):
                num_str = '0' + num_str if num_str.startswith('.') else '-0' + num_str[1:]
            
            # Проверяем пустую строку или только минус
            if not num_str or num_str == '-':
                return Decimal('0')
            
            # Преобразуем в Decimal
            num = Decimal(num_str)
            
            # Проверяем диапазон
            if num < self.MIN_VALUE or num > self.MAX_VALUE:
                return None
            
            return num
        except (InvalidOperation, ValueError):
            return None
    
    def format_number(self, num):
        """Форматирование числа для отображения"""
        if num is None:
            return "Ошибка"
        
        # Преобразуем в строку
        num_str = str(num)
        
        # Убираем незначащие нули в дробной части
        if '.' in num_str:
            integer_part, fractional_part = num_str.split('.')
            # Убираем незначащие нули
            fractional_part = fractional_part.rstrip('0')
            if fractional_part:
                num_str = integer_part + '.' + fractional_part
            else:
                num_str = integer_part
        
        return num_str
    
    def calculate(self):
        """Выполнение вычислений"""
        # Получаем числа
        num1_str = self.num1_var.get()
        num2_str = self.num2_var.get()
        
        # Проверяем, что оба числа введены
        if not num1_str or not num2_str:
            messagebox.showwarning("Внимание", "Пожалуйста, введите оба числа!")
            return
        
        # Парсим числа
        num1 = self.parse_number(num1_str)
        num2 = self.parse_number(num2_str)
        
        if num1 is None or num2 is None:
            messagebox.showerror("Ошибка", "Одно или оба числа недопустимы или выходят за диапазон!")
            return
        
        # Получаем операцию
        operation = self.operation_var.get()
        
        # Выполняем операцию
        try:
            if operation == "+":
                result = num1 + num2
            else:  # operation == "-"
                result = num1 - num2
            
            # Проверяем результат на переполнение
            if result < self.MIN_VALUE or result > self.MAX_VALUE:
                self.result_var.set("ПЕРЕПОЛНЕНИЕ")
                return
            
            # Форматируем и отображаем результат
            result_str = self.format_number(result)
            self.result_var.set(result_str)
            
        except Exception as e:
            messagebox.showerror("Ошибка", f"Ошибка при вычислении: {str(e)}")
    
    def clear_all(self):
        """Очистка всех полей"""
        self.num1_var.set("")
        self.num2_var.set("")
        self.result_var.set("")
        self.operation_var.set("+")
        self.num1_entry.focus_set()

def main():
    root = tk.Tk()
    app = CalculatorApp(root)
    
    # Глобальные горячие клавиши (работают везде в приложении)
    def global_copy(event):
        widget = root.focus_get()
        if isinstance(widget, tk.Entry):
            if widget.selection_present():
                selected_text = widget.selection_get()
                root.clipboard_clear()
                root.clipboard_append(selected_text)
            return "break"
    
    def global_paste(event):
        widget = root.focus_get()
        if isinstance(widget, tk.Entry):
            try:
                clipboard_text = root.clipboard_get()
            except:
                return "break"
            
            # Очистка текста
            cleaned = re.sub(r'[^\d\-,.]', '', clipboard_text)
            cleaned = cleaned.replace(',', '.')
            
            if cleaned.count('-') > 1:
                cleaned = cleaned[0] + cleaned[1:].replace('-', '')
            
            if cleaned.count('.') > 1:
                parts = cleaned.split('.')
                cleaned = parts[0] + '.' + ''.join(parts[1:])
            
            # Вставка
            current_text = widget.get()
            if widget.selection_present():
                start = widget.index("sel.first")
                end = widget.index("sel.last")
                new_text = current_text[:start] + cleaned + current_text[end:]
                widget.delete(0, tk.END)
                widget.insert(0, new_text)
                widget.icursor(start + len(cleaned))
            else:
                cursor_pos = widget.index(tk.INSERT)
                new_text = current_text[:cursor_pos] + cleaned + current_text[cursor_pos:]
                widget.delete(0, tk.END)
                widget.insert(0, new_text)
                widget.icursor(cursor_pos + len(cleaned))
            
            # Вызываем валидацию
            if widget == app.num1_entry:
                app.validate_input(app.num1_entry)
            elif widget == app.num2_entry:
                app.validate_input(app.num2_entry)
            
            return "break"
    
    # Регистрируем глобальные обработчики по кодам клавиш
    # Это работает независимо от раскладки
    root.bind_all('<Control-Key-c>', global_copy)
    root.bind_all('<Control-Key-C>', global_copy)
    root.bind_all('<Control-Key-v>', global_paste)
    root.bind_all('<Control-Key-V>', global_paste)
    
    # Альтернативный вариант - проверка по коду клавиши
    root.bind_all('<Control-KeyPress>', lambda e: handle_ctrl_keypress(e, root, app))
    
    root.mainloop()

def handle_ctrl_keypress(event, root, app):
    """Обработка Ctrl+клавиша по коду"""
    if event.state & 0x4:  # Ctrl нажат
        keycode = event.keycode
        
        # Копирование: C (67) в любой раскладке
        if keycode == 67:
            widget = root.focus_get()
            if isinstance(widget, tk.Entry):
                if widget.selection_present():
                    selected_text = widget.selection_get()
                    root.clipboard_clear()
                    root.clipboard_append(selected_text)
                return "break"
        
        # Вставка: V (86) в любой раскладке
        elif keycode == 86:
            widget = root.focus_get()
            if isinstance(widget, tk.Entry):
                try:
                    clipboard_text = root.clipboard_get()
                except:
                    return "break"
                
                # Очистка текста
                cleaned = re.sub(r'[^\d\-,.]', '', clipboard_text)
                cleaned = cleaned.replace(',', '.')
                
                if cleaned.count('-') > 1:
                    cleaned = cleaned[0] + cleaned[1:].replace('-', '')
                
                if cleaned.count('.') > 1:
                    parts = cleaned.split('.')
                    cleaned = parts[0] + '.' + ''.join(parts[1:])
                
                # Вставка
                current_text = widget.get()
                if widget.selection_present():
                    start = widget.index("sel.first")
                    end = widget.index("sel.last")
                    new_text = current_text[:start] + cleaned + current_text[end:]
                    widget.delete(0, tk.END)
                    widget.insert(0, new_text)
                    widget.icursor(start + len(cleaned))
                else:
                    cursor_pos = widget.index(tk.INSERT)
                    new_text = current_text[:cursor_pos] + cleaned + current_text[cursor_pos:]
                    widget.delete(0, tk.END)
                    widget.insert(0, new_text)
                    widget.icursor(cursor_pos + len(cleaned))
                
                # Вызываем валидацию
                if widget == app.num1_entry:
                    app.validate_input(app.num1_entry)
                elif widget == app.num2_entry:
                    app.validate_input(app.num2_entry)
                
                return "break"

if __name__ == "__main__":
    main()